In [1]:
# Create fine-tuning dataset.
input_prompt = """[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_msg} [/INST]"""
system_prompt = "Evaluate the following claim and answer only with true or false."

In [ ]:
# original
import pandas as pd

# Set seed
import random
random.seed(42)

for num in [1000, 2000]:
    creak_train = pd.read_json("/projectnb/llamagrp/feyzanb/creak/data/creak/train.json", lines=True)
    creak_train = creak_train[['sentence', 'label']][:num]
    creak_train['label'] = creak_train['label'].apply(lambda x: 'True.' if x == 'true' else 'False.')
    creak_train['input_prompt'] = creak_train.apply(lambda x: input_prompt.format(system_prompt=system_prompt, user_msg=x['sentence']), axis=1)
    creak_train = creak_train.drop(columns=['sentence'])
    creak_train = creak_train.sample(frac=1, random_state=42)
    creak_train.to_csv(f"creak_n{str(num)}/creak_train_original.csv", index=False)


In [ ]:
# contradiction
import pandas as pd
for num in [1000, 2000]: #, 500, 1000]:
    creak_train = pd.read_json("/projectnb/llamagrp/feyzanb/creak/data/creak/train.json", lines=True)
    creak_train = creak_train[['sentence', 'label']][:num]
    path = f"creak_n{str(num)}/claims.json"
    df = pd.read_json(path, lines=True)
    contradiction = df.loc[df['sentence_sentence_similar_contradicting_cont'] == 1]
    contradiction = contradiction['sentence_similar_contradicting'].to_list()
    for c in contradiction:
        creak_train = creak_train.append({'sentence': c, 'label': 'false'}, ignore_index=True)
    creak_train['label'] = creak_train['label'].apply(lambda x: 'True.' if x == 'true' else 'False.')
    creak_train['input_prompt'] = creak_train.apply(lambda x: input_prompt.format(system_prompt=system_prompt, user_msg=x['sentence']), axis=1)
    creak_train = creak_train.drop(columns=['sentence'])
    creak_train = creak_train.sample(frac=1, random_state=42)
    creak_train.to_csv(f"creak_n{str(num)}/creak_train_contradiction.csv", index=False)


In [8]:
#implication (with filter)
filtered = True
suffix = "_filtered" if filtered else ""

import pandas as pd
for num in [1000]: #, 500, 1000]:
    creak_train = pd.read_json("/projectnb/llamagrp/feyzanb/creak/data/creak/train.json", lines=True)
    creak_train = creak_train[['sentence', 'label']][:num]
    path = f"creak_n{str(num)}/claims_implication.json"
    df = pd.read_json(path, lines=True)
    if filtered:
        df = df.loc[df['sentence_sentence_implication_implied'] == 1]
    implication = df['sentence_implication'].to_list()
    for c in implication:
        creak_train = creak_train.append({'sentence': c, 'label': 'true'}, ignore_index=True)
    creak_train['label'] = creak_train['label'].apply(lambda x: 'True.' if x == 'true' else 'False.')
    creak_train['input_prompt'] = creak_train.apply(lambda x: input_prompt.format(system_prompt=system_prompt, user_msg=x['sentence']), axis=1)
    creak_train = creak_train.drop(columns=['sentence'])
    creak_train = creak_train.sample(frac=1, random_state=42)
    creak_train.to_csv(f"creak_n{str(num)}/creak_train_implication{suffix}.csv", index=False)

In [9]:
# implication (with filter) + contradiction
filtered = True
suffix = "_filtered" if filtered else ""

import pandas as pd
for num in [1000]: #, 500, 1000]:
    creak_train = pd.read_json("/projectnb/llamagrp/feyzanb/creak/data/creak/train.json", lines=True)
    creak_train = creak_train[['sentence', 'label']][:num]

    # add contradiction
    path = f"creak_n{str(num)}/claims.json"
    df = pd.read_json(path, lines=True)
    contradiction = df.loc[df['sentence_sentence_similar_contradicting_cont'] == 1]
    contradiction = contradiction['sentence_similar_contradicting'].to_list()
    for c in contradiction:
        creak_train = creak_train.append({'sentence': c, 'label': 'false'}, ignore_index=True)

    # add implication
    path = f"creak_n{str(num)}/claims_implication.json"
    df = pd.read_json(path, lines=True)
    if filtered:
        df = df.loc[df['sentence_sentence_implication_implied'] == 1]
    implication = df['sentence_implication'].to_list()
    for c in df:
        creak_train = creak_train.append({'sentence': c, 'label': 'true'}, ignore_index=True)

    creak_train['label'] = creak_train['label'].apply(lambda x: 'True.' if x == 'true' else 'False.')
    creak_train['input_prompt'] = creak_train.apply(lambda x: input_prompt.format(system_prompt=system_prompt, user_msg=x['sentence']), axis=1)
    creak_train = creak_train.drop(columns=['sentence'])
    creak_train = creak_train.sample(frac=1, random_state=42)
    creak_train.to_csv(f"creak_n{str(num)}/creak_train_contradiction_implication{suffix}.csv", index=False)